# Lab 5: Cross-Validation and the Bootstrap

Kingsley Ye  
AAE 722 Machine Learning  
Lab5



## Import Required Libraries


In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)
from sklearn.model_selection import (train_test_split, cross_validate, KFold)
from sklearn.base import clone
from ISLP.models import sklearn_sm
from functools import partial
import warnings
warnings.filterwarnings('ignore')


## 1. Validation Set Approach

**Question:** Use the Boston dataset from the ISLP library to predict medv (median home value) using lstat (percentage of lower status population). Split the data into training and validation sets with exactly 253 observations in each set using random_state=42 for reproducibility. Fit polynomial models of degrees 1, 2, 3, and 4 using the training set, calculate validation MSE for each polynomial degree, and report your results as a numpy array with exactly 4 elements, rounded to 2 decimal places.


In [3]:
# Load Boston dataset
Boston = load_data('Boston')
print(f"Boston dataset shape: {Boston.shape}")
print(f"Total observations: {Boston.shape[0]}")

# Split data into training and validation sets (253 observations each)
Boston_train, Boston_valid = train_test_split(Boston, 
                                            test_size=253, 
                                            random_state=42)

print(f"Training set size: {Boston_train.shape[0]}")
print(f"Validation set size: {Boston_valid.shape[0]}")


Boston dataset shape: (506, 13)
Total observations: 506
Training set size: 253
Validation set size: 253


In [4]:
# Function to evaluate MSE for polynomial models
def evalMSE(terms, response, train, test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]
    
    X_test = mm.transform(test)
    y_test = test[response]
    
    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)
    
    return np.mean((y_test - test_pred)**2)

# Calculate validation MSE for polynomial degrees 1, 2, 3, 4
MSE = np.zeros(4)
for idx, degree in enumerate(range(1, 5)):
    MSE[idx] = evalMSE([poly('lstat', degree)], 
                       'medv',
                       Boston_train,
                       Boston_valid)

# Round to 2 decimal places
MSE_rounded = np.round(MSE, 2)
print("Validation MSE for polynomial degrees 1, 2, 3, 4:")
print(MSE_rounded)
print(f"\nResults as numpy array: {MSE_rounded}")


Validation MSE for polynomial degrees 1, 2, 3, 4:
[38.51 30.84 29.22 27.75]

Results as numpy array: [38.51 30.84 29.22 27.75]


## 2. Leave-One-Out Cross-Validation

**Question:** Using the College dataset, predict Outstate (out-of-state tuition) using Room.Board (room and board costs). Implement LOOCV for polynomial degrees 1 through 5 using the sklearn_sm wrapper with cross_validate, setting cv=n where n is the total number of observations for LOOCV. Calculate the mean cross-validation error for each polynomial degree and report your results as a numpy array rounded to 2 decimal places.


In [5]:
# Load College dataset
College = load_data('College')
print(f"College dataset shape: {College.shape}")
print(f"Total observations: {College.shape[0]}")

# Prepare data for LOOCV
X = College[['Room.Board']]
Y = College['Outstate']
n = College.shape[0]
print(f"Number of observations for LOOCV: {n}")


College dataset shape: (777, 18)
Total observations: 777
Number of observations for LOOCV: 777


In [6]:
# Implement LOOCV for polynomial degrees 1 through 5
cv_error = np.zeros(5)
H = np.array(College['Room.Board'])
M = sklearn_sm(sm.OLS)

for i, d in enumerate(range(1, 6)):
    # Create polynomial features
    X_poly = np.power.outer(H, np.arange(d+1))
    
    # Perform LOOCV (cv=n for leave-one-out)
    M_CV = cross_validate(M, X_poly, Y, cv=n)
    cv_error[i] = np.mean(M_CV['test_score'])

# Round to 2 decimal places
cv_error_rounded = np.round(cv_error, 2)
print("LOOCV errors for polynomial degrees 1, 2, 3, 4, 5:")
print(cv_error_rounded)
print(f"\nResults as numpy array: {cv_error_rounded}")


LOOCV errors for polynomial degrees 1, 2, 3, 4, 5:
[9291471.1  9255509.68 9263314.52 9269406.45 9300815.88]

Results as numpy array: [9291471.1  9255509.68 9263314.52 9269406.45 9300815.88]


## 3. K-Fold Cross-Validation Comparison

**Question:** Continue with the College dataset from Question 2, but now compare 5-fold and 10-fold cross-validation using polynomial degrees 1, 2, and 3 only. For 5-fold CV use KFold(n_splits=5, shuffle=True, random_state=123) and for 10-fold CV use KFold(n_splits=10, shuffle=True, random_state=123). Calculate mean CV errors for each degree and each fold configuration, then report the difference between 5-fold and 10-fold CV errors for each polynomial degree, rounding all results to 3 decimal places.


In [7]:
# Set up K-Fold cross-validation strategies
cv_5fold = KFold(n_splits=5, shuffle=True, random_state=123)
cv_10fold = KFold(n_splits=10, shuffle=True, random_state=123)

# Calculate CV errors for 5-fold and 10-fold
cv_error_5fold = np.zeros(3)
cv_error_10fold = np.zeros(3)

for i, d in enumerate(range(1, 4)):  # degrees 1, 2, 3
    # Create polynomial features
    X_poly = np.power.outer(H, np.arange(d+1))
    
    # 5-fold CV
    M_CV_5 = cross_validate(M, X_poly, Y, cv=cv_5fold)
    cv_error_5fold[i] = np.mean(M_CV_5['test_score'])
    
    # 10-fold CV
    M_CV_10 = cross_validate(M, X_poly, Y, cv=cv_10fold)
    cv_error_10fold[i] = np.mean(M_CV_10['test_score'])

# Calculate differences
differences = cv_error_5fold - cv_error_10fold

# Round to 3 decimal places
cv_error_5fold_rounded = np.round(cv_error_5fold, 3)
cv_error_10fold_rounded = np.round(cv_error_10fold, 3)
differences_rounded = np.round(differences, 3)

print("5-fold CV errors for degrees 1, 2, 3:")
print(cv_error_5fold_rounded)
print("\n10-fold CV errors for degrees 1, 2, 3:")
print(cv_error_10fold_rounded)
print("\nDifferences (5-fold - 10-fold) for degrees 1, 2, 3:")
print(differences_rounded)
print(f"\nResults as numpy array: {differences_rounded}")


5-fold CV errors for degrees 1, 2, 3:
[9377937.163 9345970.983 9385687.251]

10-fold CV errors for degrees 1, 2, 3:
[9315542.59  9281464.281 9293767.868]

Differences (5-fold - 10-fold) for degrees 1, 2, 3:
[62394.573 64506.702 91919.384]

Results as numpy array: [62394.573 64506.702 91919.384]


## 4. Bootstrap Standard Errors

**Question:** Use the Default dataset to estimate the standard error of the correlation coefficient between balance and income using bootstrap resampling. Create a function that computes the correlation coefficient between balance and income, implement bootstrap resampling with B=2000 iterations using random_state=456 for the random number generator, and calculate the bootstrap standard error of the correlation coefficient. Also compute the actual correlation coefficient on the full dataset and round both results to 4 decimal places.


In [8]:
# Load Default dataset
Default = load_data('Default')
print(f"Default dataset shape: {Default.shape}")
print("\nFirst few rows:")
print(Default.head())


Default dataset shape: (10000, 4)

First few rows:
  default student      balance        income
0      No      No   729.526495  44361.625074
1      No     Yes   817.180407  12106.134700
2      No      No  1073.549164  31767.138947
3      No      No   529.250605  35704.493935
4      No      No   785.655883  38463.495879


In [9]:
# Function to compute correlation coefficient
def corr_func(D, idx):
    return np.corrcoef(D['balance'].loc[idx], D['income'].loc[idx])[0, 1]

# Bootstrap function for standard error
def boot_SE(func, D, n=None, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    
    for _ in range(B):
        idx = rng.choice(D.index, n, replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    
    return np.sqrt(second_ / B - (first_ / B)**2)

# Calculate actual correlation coefficient on full dataset
actual_corr = np.corrcoef(Default['balance'], Default['income'])[0, 1]

# Calculate bootstrap standard error
bootstrap_se = boot_SE(corr_func, Default, B=2000, seed=456)

# Round to 4 decimal places
actual_corr_rounded = np.round(actual_corr, 4)
bootstrap_se_rounded = np.round(bootstrap_se, 4)

print(f"Actual correlation coefficient: {actual_corr_rounded}")
print(f"Bootstrap standard error: {bootstrap_se_rounded}")
print(f"\nResults: correlation = {actual_corr_rounded}, SE = {bootstrap_se_rounded}")


Actual correlation coefficient: -0.1522
Bootstrap standard error: 0.0098

Results: correlation = -0.1522, SE = 0.0098


## 5. Bootstrap vs. Traditional Standard Errors

**Question:** Using the Wage dataset, fit both linear and polynomial (degree 2) regression models to predict wage using age. For each model calculate bootstrap standard errors for all coefficients (B=1500, seed=789) and traditional OLS standard errors using summarize(), then compare the ratio of bootstrap SE to OLS SE for each coefficient. Report results for the quadratic model only (3 coefficients: intercept, age, age²), rounding all ratios to 3 decimal places.


In [10]:
# Load Wage dataset
Wage = load_data('Wage')
print(f"Wage dataset shape: {Wage.shape}")
print("\nFirst few rows:")
print(Wage.head())


Wage dataset shape: (3000, 11)

First few rows:
   year  age            maritl      race        education              region  \
0  2006   18  1. Never Married  1. White     1. < HS Grad  2. Middle Atlantic   
1  2004   24  1. Never Married  1. White  4. College Grad  2. Middle Atlantic   
2  2003   45        2. Married  1. White  3. Some College  2. Middle Atlantic   
3  2003   43        2. Married  3. Asian  4. College Grad  2. Middle Atlantic   
4  2005   50       4. Divorced  1. White       2. HS Grad  2. Middle Atlantic   

         jobclass          health health_ins   logwage        wage  
0   1. Industrial       1. <=Good      2. No  4.318063   75.043154  
1  2. Information  2. >=Very Good      2. No  4.255273   70.476020  
2   1. Industrial       1. <=Good     1. Yes  4.875061  130.982177  
3  2. Information  2. >=Very Good     1. Yes  5.041393  154.685293  
4  2. Information       1. <=Good     1. Yes  4.318063   75.043154  


In [11]:
# Function to bootstrap OLS coefficients
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

# Fit quadratic model (degree 2)
quad_model = MS([poly('age', 2, raw=True)])
quad_func = partial(boot_OLS, quad_model, 'wage')

# Calculate bootstrap standard errors for quadratic model
bootstrap_se_quad = boot_SE(quad_func, Wage, B=1500, seed=789)

# Calculate traditional OLS standard errors for quadratic model
X_quad = quad_model.fit_transform(Wage)
Y = Wage['wage']
M_quad = sm.OLS(Y, X_quad).fit()
traditional_se_quad = summarize(M_quad)['std err']

# Calculate ratios (bootstrap SE / OLS SE)
ratios = bootstrap_se_quad / traditional_se_quad

# Round to 3 decimal places
ratios_rounded = np.round(ratios, 3)

print("Bootstrap standard errors for quadratic model:")
print(bootstrap_se_quad)
print("\nTraditional OLS standard errors for quadratic model:")
print(traditional_se_quad)
print("\nRatios (Bootstrap SE / OLS SE) for quadratic model:")
print(ratios_rounded)
print(f"\nResults as numpy array: {ratios_rounded}")


Bootstrap standard errors for quadratic model:
intercept                           6.138718
poly(age, degree=2, raw=True)[0]    0.313841
poly(age, degree=2, raw=True)[1]    0.003725
dtype: float64

Traditional OLS standard errors for quadratic model:
intercept                           8.190
poly(age, degree=2, raw=True)[0]    0.389
poly(age, degree=2, raw=True)[1]    0.004
Name: std err, dtype: float64

Ratios (Bootstrap SE / OLS SE) for quadratic model:
intercept                           0.750
poly(age, degree=2, raw=True)[0]    0.807
poly(age, degree=2, raw=True)[1]    0.931
dtype: float64

Results as numpy array: intercept                           0.750
poly(age, degree=2, raw=True)[0]    0.807
poly(age, degree=2, raw=True)[1]    0.931
dtype: float64


## Summary

1. **Validation Set Approach**: Demonstrated how to split data and evaluate polynomial models of different degrees
2. **Leave-One-Out Cross-Validation**: Implemented LOOCV using sklearn's cross_validate function
3. **K-Fold Cross-Validation**: Compared 5-fold and 10-fold CV strategies
4. **Bootstrap Standard Errors**: Estimated standard errors for correlation coefficients using bootstrap resampling
5. **Bootstrap vs. Traditional SE**: Compared bootstrap and traditional standard errors for regression coefficients
